In [138]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [139]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1495,tt0455590,6000000,48027970,The Last King of Scotland,Forest Whitaker|James McAvoy|Simon McBurney|Gi...,Kevin Macdonald,Charming. Magnetic. Murderous.,"Young Scottish doctor, Nicholas Garrigan decid...",121,Drama,Fox Searchlight Pictures|Cowboy Films|DNA Film...,9/1/2006,7.3,2006
504,tt0213149,140000000,449220945,Pearl Harbor,Ben Affleck|Josh Hartnett|Kate Beckinsale|Cuba...,Michael Bay,"December 7, 1941 - A day that shall live in in...",The lifelong friendship between Rafe McCawley ...,183,History|Romance|War,Jerry Bruckheimer Films|Touchstone Pictures,5/21/2001,6.6,2001
607,tt0795421,52000000,609841637,Mamma Mia!,Meryl Streep|Pierce Brosnan|Amanda Seyfried|Co...,Phyllida Lloyd,Take a trip down the aisle you'll never forget,"Set on an idyllic Greek island, the plot serve...",108,Comedy|Romance,Universal Pictures|Internationale Filmprodukti...,6/30/2008,6.2,2008
202,tt2106361,50000000,160602194,Into the Storm,Richard Armitage|Sarah Wayne Callies|Matt Wals...,Steven Quale,Prepare to go,The town of Silverton is in one day destroyed ...,89,Action|Thriller,New Line Cinema|Village Roadshow Pictures|Brok...,8/6/2014,5.7,2014
1236,tt1486834,11000000,7847000,What If,Daniel Radcliffe|Zoe Kazan|Rafe Spall|Megan Pa...,Michael Dowse,...being friends has its benefits?,"Wallace, a medical school dropout, has been re...",93,Drama|Comedy|Romance,Fastnet Films|Caramel Film|No Trace Camping,9/7/2013,6.7,2013


In [95]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
 14  profit                1889 non-null   int64  
dtypes: float64(1), int64(

# Предобработка

In [37]:
answers = {} # создадим словарь для ответов

In [142]:
# добавление колонки Прибыль (profit) как разница между сборами и бюджетом
data['profit'] = data['revenue']-data['budget']

In [222]:
# получение списка жанров
genres = []
data.genres.apply(lambda value: genres.extend(value.split('|')))
genres = list(set(genres))

In [65]:
# функция поиска часто встречающегося жанра
def findTheMostFrequentGenre():
    maxCount = 0
    maxGenre = ''
    for genre in genres:
        count = len(data[data.genres.str.contains(genre)])
        if count > maxCount:
            maxCount = count
            maxGenre = genre
    return maxGenre

In [67]:
# функция поиска жанра, который становится чаще всего прибыльным
def findTheMostProfitableGenre():
    maxCount = 0
    maxGenre = ''
    for genre in genres:
        count = len(data[(data.genres.str.contains(genre)) & (data.profit > 0)])
        if count > maxCount:
            maxCount = count
            maxGenre = genre
    return maxGenre

In [221]:
# получение списка режиссеров
directors = []
data.director.apply(lambda value: directors.extend(value.split('|')))
directors = list(set(directors))

In [76]:
# функция поиска режиссера с наибольшей суммарной кассой его фильмов
def findTheMostSuccessfullDirector():
    maxSum = 0
    luckyDirector = ''
    for director in directors:
        sum_ = data[data.director.str.contains(director)].revenue.sum()
        if sum_ > maxSum:
            maxSum = sum_
            luckyDirector = director
    return luckyDirector

In [79]:
# функция поиска режиссера, снявшего больше фильмов в жанре Action
def findDirectorWithTheMostCountActionMovie():
    maxCount = 0
    actionDirector = ''
    for director in directors:
        count = len(data[(data.genres.str.contains('Action')) & (data.director.str.contains(director))])
        if count > maxCount:
            maxCount = count
            actionDirector = director
    return actionDirector

In [223]:
# получение списка актеров
actors = []
data.cast.apply(lambda value: actors.extend(value.split('|')))
actors = list(set(actors))

In [87]:
# поиск актёра, фильмы с участием которого принесли самую большу кассу
def findActorWithMaxRevenueIn2012():
    maxRevenue = 0
    maxRevenueActor = ''
    movies2012 = data[data.release_year == 2012]
    for actor in actors:
        revenue = movies2012[movies2012.cast.str.contains(actor)].revenue.max()
        if revenue > maxRevenue:
            maxRevenue = revenue
            maxRevenueActor = actor
    return maxRevenueActor

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [38]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [9]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,5/11/2011,6.3,2011


ВАРИАНТ 2

In [0]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [39]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [12]:
# решение
data[data.runtime == data.runtime.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003


# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
# решение
data[data.runtime == data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,4/13/2011,6.8,2011


In [40]:
# ответ
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [15]:
# решение
data.runtime.mean()

109.6585494970884

In [41]:
# ответ
answers['4'] = '109.6585494970884'

# 5. Каково медианное значение длительности фильмов? 

In [16]:
# решение
data.runtime.median()

107.0

In [42]:
# ответ
answers['5'] = '107.0'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [0]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [19]:
# решение
data[data.profit == data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,12/10/2009,7.1,2009,2544505847


In [43]:
# ответ
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [21]:
# решение
data[data.profit == data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090


In [44]:
# ответ
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [25]:
# решение
data[data.profit > 0].profit.count()

1478

In [45]:
# ответ
answers['8'] = '1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
# решение
data[data.revenue == data[data.release_year == 2008].revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,7/16/2008,8.1,2008,816921825


In [46]:
# ответ
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [33]:
# решение
data[data.profit == data[(data.release_year >= 2012) & (data.release_year <= 2014)].profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,7/3/2013,6.0,2013,-165710090


In [47]:
# ответ
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [69]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# вариант 1
# составление списка жанров genres, встречающихся data см блок вспомогательных функций
# вызов функции countingFilmByGenres для поиска часто встречающегося жанра в data (см блок вспомогательных функций)
findTheMostFrequentGenre()

'Drama'

In [70]:
# ответ
answers['11'] = 'Drama'

ВАРИАНТ 2

In [216]:
# вернулся после решения после окончания остальных задач, зная теперь об альтернативный вариантах
data11=data.copy()
# приведем данные 1) выделяем жанры
data11['genres'] = data11['genres'].apply(lambda x: x.split('|'))
# разворачиваем таблицу по жанрам
data11_genres=data11.explode('genres')
# группируем фильмы по жанру с подсчетом количества
data11_genres.groupby(['genres'])['genres'].value_counts(ascending=False).sort_values(ascending=False)

genres           genres         
Drama            Drama              782
Comedy           Comedy             683
Thriller         Thriller           596
Action           Action             582
Adventure        Adventure          415
Crime            Crime              315
Romance          Romance            308
Family           Family             260
Science Fiction  Science Fiction    248
Fantasy          Fantasy            222
Horror           Horror             176
Mystery          Mystery            168
Animation        Animation          139
Music            Music               64
History          History             62
War              War                 58
Western          Western             19
Documentary      Documentary          8
Foreign          Foreign              2
Name: genres, dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [71]:
# вызов функции findTheMostProfitableGenre для поиска часто встречающегося жанра в data (см блок вспомогательных функций)
findTheMostProfitableGenre()

'Drama'

In [72]:
# ответ
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [77]:
# вызов функции поиска режиссера с самыми большими суммарными кассовыми сборами
findTheMostSuccessfullDirector()

'Peter Jackson'

In [78]:
# ответ
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [80]:
#решение
findDirectorWithTheMostCountActionMovie()

'Robert Rodriguez'

In [81]:
# ответ
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [88]:
# решение
findActorWithMaxRevenueIn2012()

'Chris Hemsworth'

In [99]:
# ответ
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [97]:
# дойдя до 16 вопроса, понял, что наверно использование просто функций
# - это не самый лучший вариант использования всех инструментов Phyton
# полез в слак чтобы понять что делает народ, так что решение по данному вопросу 
# это результат подглядывания), дальнейшие задачи делал уже без оглядки на слак
data16=data.copy()
# приведем данные 1) выделяем актеров
data16['cast'] = data16['cast'].apply(lambda x: x.split('|'))
# разворачиваем таблицу по актерам
data_cast=data16.explode('cast')
# фильруем данные по бюджету (выше среднего) и стаем количество таких фильмов по каждому актёру
data_cast.loc[data_cast['budget'] > data_cast['budget'].mean()]['cast'].value_counts(ascending=False)

Matt Damon              18
Adam Sandler            17
Angelina Jolie          16
Samuel L. Jackson       15
Eddie Murphy            15
                        ..
Elizabeth Olsen          1
Eric Roberts             1
Lou Diamond Phillips     1
Sarita Choudhury         1
Richard Briers           1
Name: cast, Length: 1505, dtype: int64

In [100]:
# ответ
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [108]:
# решение
data17 = data.copy()
# приведем данные 1) выделяем жанры
data17['genres'] = data17['genres'].apply(lambda x: x.split('|'))
# разворачиваем таблицу по жанрам
data_genres_17=data17.explode('genres')
# фильтруем данные по актеру 'Nicolas Cage' и считаем количество фильмов по жанру
data_genres_17.loc[data17.cast.str.contains('Nicolas Cage')]['genres'].value_counts(ascending=False)

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Animation           3
Family              3
History             2
Romance             1
Horror              1
War                 1
Name: genres, dtype: int64

In [109]:
# ответ
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [225]:
# решение
# фильтрация фильмов по компании 'Paramount Pictures' и сортировки по ПРибыли
data.loc[data['production_companies'].str.contains('Paramount Pictures')].sort_values(by='profit', ascending=True).head(1)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
925,tt0267626,100000000,35168966,K-19: The Widowmaker,Harrison Ford|Liam Neeson|Peter Sarsgaard|Joss...,Kathryn Bigelow,Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,Thriller|Drama|History,Paramount Pictures|Intermedia Films|National G...,7/19/2002,6.0,2002,-64831034


In [132]:
# ответ
answers['18'] = 'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [136]:
# решение
# группировка по году релиза и суммирование по кассе с сортировкой
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

In [204]:
# ответ
answers['19'] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [228]:
# решение
data20 = data.copy()
# фильтрация по компании 'Warner Bros' с дальнейшей группировкой по году релиза и подсчетом суммы прибыли
data20.loc[data20['production_companies'].str.contains('Warner Bros')].groupby(['release_year'])['profit'].sum().sort_values(ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

In [146]:
# ответ
answers['20'] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [227]:
# решение
data21 = data.copy()
# приведем данные 1) выделим месяц из даты релиза
data21['release_date'] = data21['release_date'].apply(lambda x: x.split('/')[0])
# сгруппируем данные по месяцу и выделим общее количество в каждом месяце
data21.groupby(['release_date'])['release_date'].count().sort_values(ascending=False)

release_date
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_date, dtype: int64

In [161]:
# ответ
answers['21'] = '9'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [169]:
# решение
data22 = data.copy()
# приведем данные 1) выделим месяц из даты релиза
data22['release_date'] = data22['release_date'].apply(lambda x: x.split('/')[0])
# отфильтруем летние месяцы и посчитаем количество
data22[(data22['release_date']=='6') | (data22['release_date']=='7') | (data22['release_date']=='8')]['release_date'].count()

450

In [170]:
# ответ
answers['22'] = '450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [177]:
data23 = data.copy()
# приведем данные 1) выделим месяц из даты релиза
data23['release_date'] = data23['release_date'].apply(lambda x: x.split('/')[0])
# отфильтруем зимние месяцы
data23 = data23[(data23['release_date']=='12') | (data23['release_date']=='1') | (data23['release_date']=='2')]
# приведем данные 2) выделим режиссеров
data23_winter['director'] = data23['director'].apply(lambda x: x.split('|'))
# развернем данные относительно режиссеров
data23_winter_director = data23_winter.explode('director')
# сгруппируем фильмы по режиссеру с подсчетом количества
data23_winter_director.groupby(['director'])['director'].count().sort_values(ascending=False)

director
Peter Jackson            7
Steven Soderbergh        6
Clint Eastwood           6
Martin Scorsese          4
Adam Shankman            4
                        ..
Matt Bettinelli-Olpin    1
Matt Reeves              1
Matthew O'Callaghan      1
Matthew Vaughn           1
Jon Avnet                1
Name: director, Length: 358, dtype: int64

In [178]:
# ответ
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [226]:
# решение
data24 = data.copy()
# приведем данные 1) выделим компании
data24['production_companies'] = data24['production_companies'].apply(lambda x: x.split('|'))
# приведем данные 2) посчитаем для каждого фильма длину названия
data24['original_title'] = data24['original_title'].apply(lambda x: len(x))
# развернем таблицу относительно компании
data24_PC = data24.explode('production_companies')
# сгруппируем фильмы по компаниям с функцией средней длинны названия
data24_PC.groupby(['production_companies'])['original_title'].mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: original_title, Length: 1771, dtype: float64

In [206]:
# ответ
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [186]:
# решение
data25 = data.copy()
# приведем данные 1) выделим компании
data25['production_companies'] = data25['production_companies'].apply(lambda x: x.split('|'))
# приведем данные 2) посчитаем количество слов в описании предполагая 
# что все слов разеляются пробелом (думаю здесь такого разделения будет достаточно)
data25['overview'] = data25['overview'].apply(lambda x: len(x.split(' ')))

# развернем таблицу относительно компании
data25_PC = data25.explode('production_companies')

# сгруппируем фильмы по компаниям с функцией средней длинны текса в описании
data25_PC.groupby(['production_companies'])['overview'].mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: overview, Length: 1771, dtype: float64

In [207]:
# ответ
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [197]:
# решение
data26 = data.copy()
# отсортируем фильмы по убыванию vote_average и возьмём первый 1% от общего числа
data26.sort_values(by='vote_average', ascending=False).head(int(len(data) / 100))['original_title'].values

array(['The Dark Knight', 'Interstellar', 'The Imitation Game',
       'Inside Out', 'Room', 'The Wolf of Wall Street', 'Gone Girl',
       '12 Years a Slave', 'Guardians of the Galaxy',
       'The Lord of the Rings: The Return of the King', 'Memento',
       'Inception', 'The Pianist', 'The Grand Budapest Hotel', 'Her',
       'Spotlight', 'Big Hero 6', 'The Fault in Our Stars'], dtype=object)

In [208]:
# ответ
answers['26'] = ['The Dark Knight', 'Interstellar', 'The Imitation Game',
       'Inside Out', 'Room', 'The Wolf of Wall Street', 'Gone Girl',
       '12 Years a Slave', 'Guardians of the Galaxy',
       'The Lord of the Rings: The Return of the King', 'Memento',
       'Inception', 'The Pianist', 'The Grand Budapest Hotel', 'Her',
       'Spotlight', 'Big Hero 6', 'The Fault in Our Stars']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [220]:
# решение
data27 = data.copy()
data27['cast'] = data27['cast'].apply(lambda x: x.split('|'))
data27_cast = data27.explode('cast')
data27_cast

# не сделано (ответ выбрал случайно), не смог найти решения

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
0,tt0369610,150000000,1513528810,Jurassic World,Bryce Dallas Howard,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
0,tt0369610,150000000,1513528810,Jurassic World,Irrfan Khan,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
0,tt0369610,150000000,1513528810,Jurassic World,Vincent D'Onofrio,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
0,tt0369610,150000000,1513528810,Jurassic World,Nick Robinson,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015,1363528810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1888,tt0163676,15000000,5217498,The In Crowd,Susan Ward,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000,-9782502
1888,tt0163676,15000000,5217498,The In Crowd,Lori Heuring,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000,-9782502
1888,tt0163676,15000000,5217498,The In Crowd,Matthew Settle,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000,-9782502
1888,tt0163676,15000000,5217498,The In Crowd,Nathan Bexton,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000,-9782502


ВАРИАНТ 2

# Submission

In [229]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': '109.6585494970884',
 '5': '107.0',
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': '1478',
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': '9',
 '22': '450',
 '23': 'Peter Jackson',
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  'Big Hero 6',
  'The Fault in O

In [230]:
# и убедиться что ни чего не пропустил)
len(answers)

26